In [2]:
import xarray as xr
from workflow.scripts.utils import regrid_global

In [8]:
ds = xr.open_dataset(snakemake.input.forcing)
antro_aod = xr.open_dataset(snakemake.input.aod)
vname = snakemake.wildcards.vName

In [4]:
if ds.attrs.get('regridded', 'False') == 'False':
    if not ds.cf.bounds.get('lon',None):
        ds = ds.cf.add_bounds(['lon','lat'])

    grid_params = snakemake.config['regrid_params']
    grid_path = grid_params.get('grid_path',None)
    method=grid_params.get('method','conservative')
    if grid_path:
        out_grid = xr.open_dataset(grid_path)
        ds = regrid_global(ds, out_grid, method=method)
    elif grid_params.get('dxdy',None):
        dxdy = grid_params['dxdy']
        ds = regrid_global(ds, lon=dxdy[0], lat=dxdy[1], method=method)
    else:
        print('No outgrid provided!')


In [7]:
snakemake.config['regrid_params']

In [12]:
ds.attrs.get('regridded', 'False') == 'False'

In [37]:
out_ds = ds.copy()

out_ds = out_ds.assign({f'{snakemake.wildcards.vName}_aod': ds[vname]/antro_aod['od550aer'].values})

out_ds['ERFt_aod'].attrs['units'] = 'W m-2 AOD-1'
out_ds['ERFt_aod'].attrs['long_name'] = 'Annual mean forcing efficiency at the top of the atmosphere'
out_ds = out_ds.drop(vname)

out_ds.to_netcdf(snakemake.output.outpath)